<a href="https://colab.research.google.com/github/jithinmjoison/fast_ai_from_foundations/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Autoencoder
from IPython import get_ipython
from IPython.display import display
# %%
# Autoencoder

from tensorflow.keras import layers, models, datasets, callbacks
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf # fixed import
import tensorflow.keras.backend as K


In [17]:
### Paramaters for Auto Encoder

In [18]:
IMAGE_SIZE = 32
CHANNELS = 1
BATCH_SIZE = 100
BUFFER_SIZE = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 2
EPOCHS = 3

In [19]:
### Data Prep

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [20]:
### Function to Pre-process the data

def preprocess_image(imgs):
  """
  normalise the reshape Image
  """

  imgs = imgs.astype("float32")/255.0
  imgs = np.pad(imgs, ((0,0),(2,2),(2,2)), constant_values =0.0)
  imgs = np.expand_dims(imgs,-1)
  return imgs

In [21]:
x_train = preprocess_image(x_train)
x_test = preprocess_image(x_test)

In [22]:
display(x_train)

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [23]:
### Building the autoencoder

encoder_input = layers.Input(shape =(IMAGE_SIZE, IMAGE_SIZE, CHANNELS), name = 'encoding input')

x = layers.Conv2D(32,(3,3), strides = 2, padding= 'same', activation = 'relu')(encoder_input)

x = layers.Conv2D(64,(3,3), strides = 2, padding= 'same', activation = 'relu')(x)

x = layers.Conv2D(128,(3,3), strides = 2, padding= 'same', activation = 'relu')(x)

shape_before_flattening = K.int_shape(x)[1:]

x = layers.Flatten()(x)

encoder_output = layers.Dense(EMBEDDING_DIM, name = 'encoding_output')(x)

encoder = models.Model(encoder_input, encoder_output)

encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoding input (InputLayer  [(None, 32, 32, 1)]       0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 encoding_output (Dense)     (None, 2)                 4098      
                                                             

In [26]:
### Building the Decoder
decoder_input = layers.Input(shape = (EMBEDDING_DIM,), name = 'decoding input')

x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)

x= layers.Reshape(shape_before_flattening)(x)

x= layers.Conv2DTranspose(128,(3,3), strides = 2, padding= 'same', activation = 'relu')(x)

x= layers.Conv2DTranspose(64,(3,3), strides = 2, padding= 'same', activation = 'relu')(x)

x=  layers.Conv2DTranspose(32,(3,3), strides = 2, padding= 'same', activation = 'relu')(x)

decoder_output = layers.Conv2DTranspose(CHANNELS,(3,3), strides = 1, padding= 'same', activation = 'sigmoid')(x)

decoder = models.Model(decoder_input, decoder_output)
decoder.summary()

decoder = models.Model(decoder_input, decoder_output)

decoder.summary()



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoding input (InputLayer  [(None, 2)]               0         
 )                                                               
                                                                 
 dense_1 (Dense)             (None, 2048)              6144      
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 8, 8, 128)         147584    
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 16, 16, 64)        73792     
 Transpose)                                                      
                                                           

In [27]:
### Autoencoder

autoencoder = models.Model(encoder_input, decoder(encoder(encoder_input)))
autoencoder.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoding input (InputLayer  [(None, 32, 32, 1)]       0         
 )                                                               
                                                                 
 model (Functional)          (None, 2)                 96770     
                                                                 
 model_2 (Functional)        (None, 32, 32, 1)         246273    
                                                                 
Total params: 343043 (1.31 MB)
Trainable params: 343043 (1.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
